In [3]:
import os
import sys
sys.path.append(os.path.abspath('..'))

import torch
from torch.utils.data import DataLoader
from transformer_lens import HookedTransformer
from transformer_lens import utils as tutils
from transformer_lens.evals import make_pile_data_loader, evaluate_on_dataset

from functools import partial
from datasets import load_dataset
import tqdm

from sae_lens import SparseAutoencoder
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes
from sae_lens import SparseAutoencoder, ActivationsStore

from steering.eval_utils import evaluate_completions
from steering.utils import get_activation_steering, get_sae_diff_steering, remove_sae_feats, text_to_sae_feats, top_activations
from steering.patch import generate, get_loss

from steering.utils import text_to_sae_feats, top_activations, normalise_decoder
from steering.patch import generate

import plotly.express as px

torch.set_grad_enabled(False)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HookedTransformer.from_pretrained("gemma-2b", device=device)

/Users/slava/safety/steering/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b into HookedTransformer


In [4]:
hp6 = "blocks.6.hook_resid_post"
sae6 = SparseAutoencoder.from_pretrained("gemma-2b-res-jb", hp6)
normalise_decoder(sae6)
activation_store = ActivationsStore.from_config(model, sae6.cfg)

Using Ghost Grads.


Resolving data files:   0%|          | 0/23032 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23032 [00:00<?, ?it/s]

## Anger

In [12]:
pos_text = "Anger"
neg_text = "Calm"

anger_activation_steering= get_activation_steering(model, hp6, pos_text, neg_text)
anger_activation_steering.shape

torch.Size([1, 2, 2048])

In [16]:
generate(model,
         hp6,
         prompt="I think that",
         steering_vector=anger_activation_steering,
         scale=10)

100%|██████████| 5/5 [01:17<00:00, 15.42s/it]


['I think that in some he had said her than he died!\nI once mentioned the work of Jon Jones,',
 'I think that if it weren’t for the war movie, the most interesting bit of the game, this game',
 'I think that the roguish characters could use more distinct mechanics. Blades, Serpentina, Hylia, and',
 'I think that app is shit, I actually hate how it goes like and, "get this app, what about',
 'I think that includes a lot of wisdom.\nSunday Night Football, Gore and Munster seem back in form.\n\n']

In [17]:
top_activations(text_to_sae_feats(model, sae6, hook_point=hp6, text=pos_text))

(tensor([[[24.4932, 23.9630, 23.2544, 21.0681, 20.9372, 20.6655, 20.3199,
           19.8953, 19.0510, 18.4377],
          [56.7609, 14.0818, 12.2185, 11.2966, 10.9838,  9.4307,  8.0595,
            7.1658,  7.1005,  5.8064]]]),
 tensor([[[ 3390, 15881,  5347, 16334,   556,  8704, 11785,  5624,  5396,  6877],
          [ 1062, 12753,  1213, 11968, 12167,  5915,  2491, 15173, 11912, 12312]]]))

In [15]:
model.tokenizer.vocab_size

256000